### Avaliacao Final

> Aluna: Andressa Gomes Moreira

> Dataset: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


In [1]:
# Importações
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

from natsort import natsorted

#### Análise dos Dados

In [3]:
df = pd.read_table("IMDB Dataset.csv", sep =',')
df.head()  #df[:10]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['review'][1]

#### Limpeza dos Dados

In [4]:
def limpezaDados(text):
    # Remove Tags html
    text_Tags = re.compile('<.*?>')    
    text = text_Tags.sub(r'', text)
    # Remove url
    text_url=re.compile(r"https://\S+|http://\S+")
    text=text_url.sub(r" ", text)
    # Remove pontuações
    text_pont=re.compile(r"[^\w\s]")
    text=text_pont.sub(r" ", text)
    # Remove Dígitos
    text_dig=re.compile(r"\d{1,}")
    text=text_dig.sub(r" ", text)
    # Remove caractere único
    text_caract=re.compile(r"\s\w\s")
    text=text_caract.sub(r" ",text)
    # Remove múltiplos espaços
    text_esp=re.compile(r"\s{2,}")                    
    text=text_esp.sub(r" ",text)
    
    return text.lower()

df['review'] = df['review'].apply(limpezaDados)
df['review'][1]

'a wonderful little production the filming technique is very unassuming very old time bbc fashion and gives comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is terrificly written and performed piece masterful production about one of the great master of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell murals decorating every surface are terribly well done '

#### Tratamento dos Dados

In [5]:
# Stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andressa.moreira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords

In [6]:
def remove_stopW(text):
    text_semStopw=[]
    for word in text.split():
        if word not in stopwords:
            text_semStopw.append(word)
    return " ".join(text_semStopw)

df['review']=df['review'].apply(remove_stopW)
df['review'][1]

'wonderful little production filming technique unassuming old time bbc fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seamless editing guided references williams diary entries well worth watching terrificly written performed piece masterful production one great master comedy life realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears plays knowledge senses particularly scenes concerning orton halliwell sets particularly flat halliwell murals decorating every surface terribly well done'

#### Divisão dos Dados

In [53]:
reviews = df['review'].values
labels = df['sentiment'].values
#print(labels)
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
encoded_labels
len(reviews)

50000

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector1 = []
for i in range(len(reviews)):
    tokens= nltk.tokenize.word_tokenize(reviews[i])
    tfd=TfidfVectorizer(ngram_range=(1,3),max_features=5000)
    vector=tfd.fit_transform(tokens)
    vector=vector.toarray()
    vector1.append(vector)
vector.shape

(66, 54)

In [61]:
tfd=TfidfVectorizer(ngram_range=(1,3),max_features=5000)
vector=tfd.fit_transform(reviews)
vector=vector.toarray()

In [74]:
#vector.shape
X_train, X_test, y_train, y_test = train_test_split(vector, encoded_labels, test_size = 0.2)

In [76]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=5000))
model.add(Bidirectional(LSTM(128)))
#model.add(LSTM(128))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5000, 100)         300000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              234496    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 551,009
Trainable params: 551,009
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, verbose=1,
                    steps_per_epoch = len(X_train)/256,batch_size = 256,validation_split=0.1)

Epoch 1/5


KeyboardInterrupt: 

In [71]:
print(X_train.shape)
print(y_train.shape)
print(X_train_padded.shape)

(40000, 5000)
(40000,)
(40000, 200)


In [46]:
tokens= nltk.tokenize.word_tokenize(reviews[0])
fd= nltk.probability.FreqDist(tokens)
fd.most_common(5)

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test, padding='post', maxlen=max_length)

In [47]:
fd

FreqDist({'oz': 6, 'violence': 4, 'show': 3, 'prison': 3, 'forget': 3, 'watching': 2, 'episode': 2, 'right': 2, 'first': 2, 'struck': 2, ...})

In [29]:
X_train, X_test, y_train, y_test = train_test_split(reviews, encoded_labels, test_size = 0.2)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, stratify = y_train)


In [10]:
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
vocab_size = 3000
oov_tok = ''
embedding_dim = 100
max_length = 200
trunc_type = 'post'
padding_type = 'post'

In [11]:
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train, padding='post', maxlen=max_length)
X_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test, padding='post', maxlen=max_length)

In [44]:
X_test_padded.shape

(10000, 200)

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(128)))
#model.add(LSTM(128))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 551,009
Trainable params: 551,009
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(X_train_padded, y_train, epochs=5, verbose=1,steps_per_epoch = len(X_train)/256,batch_size = 256,validation_split=0.1)

Epoch 1/5
 32/156 [=====>........................] - ETA: 6:14 - loss: 0.6798 - accuracy: 0.5753

KeyboardInterrupt: 

In [ ]:
fd= nltk.probability.FreqDist(tokens)

In [ ]:
fd.most_common(5)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12,8))
fd.plot(25)

In [ ]:
dom_simp=nltk.Text(nltk.tokenize.word_tokenize(texto))

In [ ]:
dom_simp.collocations()

In [ ]:
dom_simp.dispersion_plot(['capitu','mãe','olhos', 'seminário','amor', 'bentinho','escobar', 'ezequiel'])

In [ ]:
dom_simp